In [1]:
import folium
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import box
import numpy as np
from shapely import wkt
import plotly.express as px
import plotly.graph_objects as go
from shapely.geometry import Point
from pyproj import Proj, transform, CRS
from folium.plugins import MarkerCluster

In [2]:
import matplotlib.font_manager as fm

# 한국어 출력을 위한 폰트 설정
plt.rc('font', family='NanumGothic') 

# 마이너스 기호가 깨지는 것을 방지
plt.rcParams['axes.unicode_minus'] = False

In [3]:
# 송파소방서 소방용수
fire_water = pd.read_excel("data/(송파소방서)소방용수.xlsx")

C:\Users\1qlqj\Desktop\multicamp_semi\multicamp_semi\venv\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [4]:
grid = gpd.read_file("data/seoul_geo/seoul_geo.shp", encoding='utf-8')

In [5]:
fire_water.head()

,순번,일련번호,수리번호,공설구분,용수형식,용수구분,수압,사용구분,관할서,안전센터,도로명,건물본번,건물부번,우편번호,좌표X,좌표Y,경위도X,경위도Y
0,1,110083413,440122,사설,지상식,소화전,NaN,양호,송파소방서,잠실119안전센터,올림픽로,300.0,0.0,NaN,209279.7409,445982.3528,127.104975,37.513220
1,2,110065754,140090,사설,지상식,소화전,3.0,양호,송파소방서,현장대응단,오금로,307.0,0.0,NaN,211193.2641,444880.8784,127.126605,37.503274
2,3,110066991,200450,공설,지상식,소화전,2.5,양호,송파소방서,거여119안전센터,오금로,524.0,0.0,NaN,212996.2247,443716.5910,127.146977,37.492759
3,4,110066992,200451,공설,지상식,소화전,3.0,양호,송파소방서,거여119안전센터,오금로,550.0,0.0,NaN,213257.1350,443702.1963,127.149928,37.492625
4,5,110066993,200461,공설,지상식,소화전,3.0,양호,송파소방서,거여119안전센터,오금로,540.0,0.0,NaN,213146.0496,443705.6892,127.148671,37.492658


In [6]:
grid.head()

,id,pop,total_b,res_single,res_multi,b_over_20,EMD_CD,EMD_KOR_NM,geometry
0,1,364.0,13.0,3.0,4.0,NaN,11560111,당산동1가,"POLYGON ((126.89910 37.51987, 126.89909 37.520..."
1,2,448.0,21.0,3.0,18.0,16.0,11590108,대방동,"POLYGON ((126.92866 37.50109, 126.92866 37.501..."
2,3,426.0,2.0,NaN,2.0,NaN,11710109,장지동,"POLYGON ((127.13469 37.47576, 127.13469 37.476..."
3,4,215.0,10.0,1.0,6.0,NaN,11470103,신월동,"POLYGON ((126.84134 37.52497, 126.84133 37.525..."
4,5,52.0,17.0,3.0,NaN,13.0,11680105,삼성동,"POLYGON ((127.04400 37.51060, 127.04400 37.511..."


In [7]:
# 경도와 위도를 이용해 Point 지오메트리 생성
fire_water['geometry'] = fire_water.apply(lambda row: Point(row['경위도X'], row['경위도Y']), axis=1)

# 새로운 geometry 열을 사용해 GeoDataFrame을 다시 생성
fire_water = gpd.GeoDataFrame(fire_water, geometry='geometry')

# CRS 설정
fire_water.set_crs(epsg=4326, inplace=True)

,순번,일련번호,수리번호,공설구분,용수형식,용수구분,수압,사용구분,관할서,안전센터,도로명,건물본번,건물부번,우편번호,좌표X,좌표Y,경위도X,경위도Y,geometry
0,1,110083413,440122,사설,지상식,소화전,NaN,양호,송파소방서,잠실119안전센터,올림픽로,300.0,0.0,NaN,209279.7409,445982.3528,127.104975,37.513220,POINT (127.10498 37.51322)
1,2,110065754,140090,사설,지상식,소화전,3.0,양호,송파소방서,현장대응단,오금로,307.0,0.0,NaN,211193.2641,444880.8784,127.126605,37.503274,POINT (127.12660 37.50327)
2,3,110066991,200450,공설,지상식,소화전,2.5,양호,송파소방서,거여119안전센터,오금로,524.0,0.0,NaN,212996.2247,443716.5910,127.146977,37.492759,POINT (127.14698 37.49276)
3,4,110066992,200451,공설,지상식,소화전,3.0,양호,송파소방서,거여119안전센터,오금로,550.0,0.0,NaN,213257.1350,443702.1963,127.149928,37.492625,POINT (127.14993 37.49263)
4,5,110066993,200461,공설,지상식,소화전,3.0,양호,송파소방서,거여119안전센터,오금로,540.0,0.0,NaN,213146.0496,443705.6892,127.148671,37.492658,POINT (127.14867 37.49266)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3812,3813,110084779,640205,사설,지상식,소화전,3.0,양호,송파소방서,가락119안전센터,법원로11길,25.0,0.0,NaN,199657.8231,450752.2932,126.996127,37.556249,POINT (126.99613 37.55625)
3813,3814,110084905,140131,사설,지상식,소화전,NaN,양호,송파소방서,현장대응단,NaN,NaN,NaN,NaN,199657.8231,450752.2932,126.996127,37.556249,POINT (126.99613 37.55625)
3814,3815,110061611,300269,공설,지하식,소화전,2.8,양호,송파소방서,방이119안전센터,바람드리9길,10.0,0.0,NaN,210532.6069,448773.1813,127.119188,37.538355,POINT (127.11919 37.53835)
3815,3816,110058491,300555,공설,지하식,소화전,3.0,양호,송파소방서,방이119안전센터,올림픽로32길,36.0,18.0,NaN,209717.0360,445881.8552,127.109921,37.512310,POINT (127.10992 37.51231)


In [8]:
# 공간 조인을 수행하여 각 songpa 그리드 내의 fire_water 수를 확인 (predicate 사용)
joined = gpd.sjoin(grid, fire_water, how='left', predicate='contains')

In [9]:
joined

,id,pop,total_b,res_single,res_multi,b_over_20,EMD_CD,EMD_KOR_NM,geometry,index_right,...,관할서,안전센터,도로명,건물본번,건물부번,우편번호,좌표X,좌표Y,경위도X,경위도Y
0,1,364.0,13.0,3.0,4.0,NaN,11560111,당산동1가,"POLYGON ((126.89910 37.51987, 126.89909 37.520...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,448.0,21.0,3.0,18.0,16.0,11590108,대방동,"POLYGON ((126.92866 37.50109, 126.92866 37.501...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,426.0,2.0,NaN,2.0,NaN,11710109,장지동,"POLYGON ((127.13469 37.47576, 127.13469 37.476...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,215.0,10.0,1.0,6.0,NaN,11470103,신월동,"POLYGON ((126.84134 37.52497, 126.84133 37.525...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,52.0,17.0,3.0,NaN,13.0,11680105,삼성동,"POLYGON ((127.04400 37.51060, 127.04400 37.511...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61642,61643,NaN,NaN,NaN,NaN,NaN,11350105,상계동,"POLYGON ((127.08835 37.67661, 127.08834 37.677...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61643,61644,NaN,NaN,NaN,NaN,NaN,11500109,방화동,"POLYGON ((126.81818 37.58252, 126.81817 37.583...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61644,61645,NaN,NaN,NaN,NaN,NaN,11350102,월계동,"POLYGON ((127.06374 37.61794, 127.06374 37.618...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61645,61646,NaN,NaN,NaN,NaN,NaN,11650109,내곡동,"POLYGON ((127.06813 37.44669, 127.06813 37.447...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# 각 songpa 그리드별 fire_water 개수 집계
fire_water_count = joined['id'].value_counts().reset_index()
fire_water_count.columns = ['id', 'count']

In [11]:
fire_water_count

,id,count
0,13379,78
1,7867,9
2,11667,8
3,8505,8
4,27666,6
...,...,...
61642,20896,1
61643,20897,1
61644,20898,1
61645,20899,1


In [12]:
water = pd.merge(grid, fire_water_count, on='id', how='left')
water

,id,pop,total_b,res_single,res_multi,b_over_20,EMD_CD,EMD_KOR_NM,geometry,count
0,1,364.0,13.0,3.0,4.0,NaN,11560111,당산동1가,"POLYGON ((126.89910 37.51987, 126.89909 37.520...",1
1,2,448.0,21.0,3.0,18.0,16.0,11590108,대방동,"POLYGON ((126.92866 37.50109, 126.92866 37.501...",1
2,3,426.0,2.0,NaN,2.0,NaN,11710109,장지동,"POLYGON ((127.13469 37.47576, 127.13469 37.476...",1
3,4,215.0,10.0,1.0,6.0,NaN,11470103,신월동,"POLYGON ((126.84134 37.52497, 126.84133 37.525...",1
4,5,52.0,17.0,3.0,NaN,13.0,11680105,삼성동,"POLYGON ((127.04400 37.51060, 127.04400 37.511...",1
...,...,...,...,...,...,...,...,...,...,...
61642,61643,NaN,NaN,NaN,NaN,NaN,11350105,상계동,"POLYGON ((127.08835 37.67661, 127.08834 37.677...",1
61643,61644,NaN,NaN,NaN,NaN,NaN,11500109,방화동,"POLYGON ((126.81818 37.58252, 126.81817 37.583...",1
61644,61645,NaN,NaN,NaN,NaN,NaN,11350102,월계동,"POLYGON ((127.06374 37.61794, 127.06374 37.618...",1
61645,61646,NaN,NaN,NaN,NaN,NaN,11650109,내곡동,"POLYGON ((127.06813 37.44669, 127.06813 37.447...",1


In [13]:
water['count'] = water['count'].apply(lambda x: 6 if x >= 6 else x)

In [14]:
water['count'].value_counts()

count
1    60570
2      451
3      364
4      181
5       59
6       22
Name: count, dtype: int64

In [15]:
water.head()

,id,pop,total_b,res_single,res_multi,b_over_20,EMD_CD,EMD_KOR_NM,geometry,count
0,1,364.0,13.0,3.0,4.0,NaN,11560111,당산동1가,"POLYGON ((126.89910 37.51987, 126.89909 37.520...",1
1,2,448.0,21.0,3.0,18.0,16.0,11590108,대방동,"POLYGON ((126.92866 37.50109, 126.92866 37.501...",1
2,3,426.0,2.0,NaN,2.0,NaN,11710109,장지동,"POLYGON ((127.13469 37.47576, 127.13469 37.476...",1
3,4,215.0,10.0,1.0,6.0,NaN,11470103,신월동,"POLYGON ((126.84134 37.52497, 126.84133 37.525...",1
4,5,52.0,17.0,3.0,NaN,13.0,11680105,삼성동,"POLYGON ((127.04400 37.51060, 127.04400 37.511...",1


In [16]:
# 열 이름 변경
water.rename(columns={
    'count': 'fire_water'
}, inplace=True)

In [17]:
water.head()

,id,pop,total_b,res_single,res_multi,b_over_20,EMD_CD,EMD_KOR_NM,geometry,fire_water
0,1,364.0,13.0,3.0,4.0,NaN,11560111,당산동1가,"POLYGON ((126.89910 37.51987, 126.89909 37.520...",1
1,2,448.0,21.0,3.0,18.0,16.0,11590108,대방동,"POLYGON ((126.92866 37.50109, 126.92866 37.501...",1
2,3,426.0,2.0,NaN,2.0,NaN,11710109,장지동,"POLYGON ((127.13469 37.47576, 127.13469 37.476...",1
3,4,215.0,10.0,1.0,6.0,NaN,11470103,신월동,"POLYGON ((126.84134 37.52497, 126.84133 37.525...",1
4,5,52.0,17.0,3.0,NaN,13.0,11680105,삼성동,"POLYGON ((127.04400 37.51060, 127.04400 37.511...",1


In [18]:
water.to_file("data/water_grid.shp")